In [5]:
import torch
import torch.nn as nn
import numpy as np
from utilis_ForsaV2 import *
import utilis_ForsaV2
# !pip install fxpmath\
from fxpmath import Fxp
import time
word_size  = 8
frac_size = 6
utilis_ForsaV2.word_size = word_size
utilis_ForsaV2.frac_size = frac_size
# from tensorboardX import SummaryWriter
# from transforms import *
# from models_cust import *
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

Using cache found in C:\Users\huruy/.cache\torch\hub\pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
# # Uncomment to visualize it in netron locally
# # Assuming `model` is your PyTorch model
# model_path = './PretrainedModels/resnet18PretrainedIM1K.pth'

# # Save the entire model, including its architecture and trained parameters
# # torch.save(model, model_path)
# torch.jit.script(model).save(model_path)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

11689512


In [6]:
# Generate index from sorting conv1 to rearrange layers that follow
_, new_indx1 = sortFullMatrix_V2(model.conv1.weight)
# Sorting 
model.conv1.weight.data, _ = compare_sw_sort(model.conv1.weight.data, new_indx1, -3)
############ rearranging bn1
model.bn1.weight.data, _ = compare_sw_sort(model.bn1.weight, new_indx1, -1)
model.bn1.bias.data, _ = compare_sw_sort(model.bn1.bias, new_indx1, -1)
model.bn1.running_mean.data, _ = compare_sw_sort(model.bn1.running_mean, new_indx1, -1)
model.bn1.running_var.data, _ = compare_sw_sort(model.bn1.running_var, new_indx1, -1)
###############################
############## Layer 1.0 ########
# Rearranging layer1.0.conv1 along channel dimension
model.layer1[0].conv1.weight.data =model.layer1[0].conv1.weight[:, new_indx1, :, :]  
### Sorting layer1.0.conv1 along filter dimension
model.layer1[0].conv2.weight.data =model.layer1[0].conv2.weight[new_indx1, :, :, :]  
model.layer1[0].bn2.weight.data =model.layer1[0].bn2.weight[new_indx1]  
model.layer1[0].bn2.bias.data =model.layer1[0].bn2.bias[new_indx1]  
model.layer1[0].bn2.running_mean.data =model.layer1[0].bn2.running_mean[new_indx1]  
model.layer1[0].bn2.running_var.data =model.layer1[0].bn2.running_var[new_indx1]  

###############################
############## Layer 1.1 ########
# Rearranging layer1.0.conv1 along channel dimension
model.layer1[1].conv1.weight.data =model.layer1[1].conv1.weight[:, new_indx1, :, :]  
### Sorting layer1.0.conv1 along filter dimension
model.layer1[1].conv2.weight.data =model.layer1[1].conv2.weight[new_indx1, :, :, :]  
model.layer1[1].bn2.weight.data =model.layer1[1].bn2.weight[new_indx1]  
model.layer1[1].bn2.bias.data =model.layer1[1].bn2.bias[new_indx1]  
model.layer1[1].bn2.running_mean.data =model.layer1[1].bn2.running_mean[new_indx1]  
model.layer1[1].bn2.running_var.data =model.layer1[1].bn2.running_var[new_indx1]  

###############################
############## Layer 2.0 ########
# Rearranging layer1.0.conv1 along channel dimension
model.layer2[0].conv1.weight.data =model.layer2[0].conv1.weight[:, new_indx1, :, :]  

###############################
############## Layer 2.0 ########
# Rearranging layer1.0.conv1 along channel dimension
model.layer2[0].downsample[0].weight.data =model.layer2[0].downsample[0].weight[:, new_indx1, :, :]  

# model.layer1[0].conv2.weight.data, _ = compare_sw_sort(model.layer1[0].conv2.weight, new_indx1, -3)
# model.layer1[0].bn2.weight.data, _ = compare_sw_sort(model.layer1[0].bn2.weight, new_indx1, -1)
# model.layer1[0].bn2.bias.data, _ = compare_sw_sort(model.layer1[0].bn2.bias, new_indx1, -1)
# model.layer1[0].bn2.running_mean.data, _ = compare_sw_sort(model.layer1[0].bn2.running_mean, new_indx1, -1)
# model.layer1[0].bn2.running_var.data, _ = compare_sw_sort(model.layer1[0].bn2.running_var, new_indx1, -1)


Switching before sorting 33858.0
Switching after sorting 27188.0
Percentage of reduction 19.69992320869514 %
Switching before sorting 156.0
Switching after sorting 123.0
Percentage of reduction 21.153846153846153 %
Switching before sorting 212.0
Switching after sorting 186.0
Percentage of reduction 12.264150943396226 %
Switching before sorting 254.0
Switching after sorting 240.0
Percentage of reduction 5.511811023622047 %
Switching before sorting 260.0
Switching after sorting 206.0
Percentage of reduction 20.76923076923077 %


# Evaluating Model output 

In [7]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
filename = 'dog.jpg'
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes

# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)


In [8]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.88462233543396
Arctic fox 0.0458049476146698
white wolf 0.044276103377342224
Pomeranian 0.0056213438510894775
Great Pyrenees 0.004652012605220079
